In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
os.chdir('C:/Users/acer/Desktop/Aries/archive')
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

In [22]:
data = []
labels = []
# We have 43 Classes
classes = 43
cur_path = os.getcwd()

In [23]:
cur_path

'C:\\Users\\acer\\Desktop\\Aries\\archive'

In [24]:
for i in range(classes):
    print(i)
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except Exception as e:
            print(e)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


In [25]:
data = np.array(data)
labels = np.array(labels)

In [27]:
# os.mkdir('training')
np.save('./training/data',data)
np.save('./training/target',labels)

In [28]:
data=np.load('./training/data.npy')
labels=np.load('./training/target.npy')

In [29]:
print(data.shape, labels.shape)

(39209, 30, 30, 3) (39209,)


In [30]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=0)

In [31]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(31367, 30, 30, 3) (7842, 30, 30, 3) (31367,) (7842,)


In [32]:
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

In [33]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
# We have 43 classes that's why we have defined 43 in the dense
model.add(Dense(43, activation='softmax'))

In [34]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
epochs = 20
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/20
981/981 [==============================] - 64s 63ms/step - loss: 1.8000 - accuracy: 0.5573 - val_loss: 0.3111 - val_accuracy: 0.9280
Epoch 2/20
981/981 [==============================] - 57s 58ms/step - loss: 0.4795 - accuracy: 0.8711 - val_loss: 0.1856 - val_accuracy: 0.9494
Epoch 3/20
981/981 [==============================] - 69s 70ms/step - loss: 0.3096 - accuracy: 0.9173 - val_loss: 0.0936 - val_accuracy: 0.9735
Epoch 4/20
981/981 [==============================] - 69s 70ms/step - loss: 0.2372 - accuracy: 0.9345 - val_loss: 0.0737 - val_accuracy: 0.9776
Epoch 5/20
981/981 [==============================] - 69s 70ms/step - loss: 0.2299 - accuracy: 0.9380 - val_loss: 0.0580 - val_accuracy: 0.9844
Epoch 6/20
981/981 [==============================] - 67s 69ms/step - loss: 0.2201 - accuracy: 0.9406 - val_loss: 0.0508 - val_accuracy: 0.9878
Epoch 7/20
981/981 [==============================] - 66s 67ms/step - loss: 0.1996 - accuracy: 0.9462 - val_loss: 0.0599 - val_accuracy:

In [36]:
def testing(testcsv):
    y_test = pd.read_csv(testcsv)
    label = y_test["ClassId"].values
    imgs = y_test["Path"].values
    data=[]
    for img in imgs:
        image = Image.open(img)
        image = image.resize((30,30))
        data.append(np.array(image))
    X_test=np.array(data)
    return X_test,label

In [37]:
X_test, label = testing('Test.csv')

In [40]:
Y_pred_prob = model.predict(X_test)
Y_pred = np.argmax(Y_pred_prob, axis=1)

395/395 [==============================] - 11s 27ms/step


In [41]:
Y_pred

array([16,  1, 38, ..., 12,  7, 10], dtype=int64)

In [42]:
from sklearn.metrics import accuracy_score
print(accuracy_score(label, Y_pred))

0.9665083135391924


In [43]:
model.save("./training/Avnish")

INFO:tensorflow:Assets written to: ./training/Avnish\assets


INFO:tensorflow:Assets written to: ./training/Avnish\assets


In [45]:
import os
os.chdir('C:/Users/acer/Desktop/Aries/archive')
from keras.models import load_model
model = load_model('./training/Avnish')

In [46]:
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

In [47]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
def test_on_img(img):
    data=[]
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
    X_test=np.array(data)
    Y_pred = model.predict_classes(X_test)
    return image,Y_pred

In [48]:
plot,prediction = test_on_img("C:/Users/acer/Desktop/Aries/archive/Test/00057.png")
s = [str(i) for i in prediction] 
a = int("".join(s)) 
print("Predicted traffic sign is: ", classes[a])
plt.imshow(plot)
plt.show()

AttributeError: 'Sequential' object has no attribute 'predict_classes'